<div style="text-align: right"><b> Ana Valentina López Chacón </b></div>
<div style="text-align: right"><b> Traducción Automática, 2024 - 2025 </b></div>

# **Portugués a Ingles: Finetuning con NLLB**

In [1]:
from google.colab import drive
drive.mount('/content/drive')
# path = '/content/drive/MyDrive/Colab Notebooks/TA/A3/'

Mounted at /content/drive


In [2]:
!pip install -q git+https://github.com/openai/whisper.git
!pip install -q datasets jiwer tqdm pandas
!pip install -q torch torchvision torchaudio

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 77.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [3]:
!pip install -q evaluate
!pip install -q transformers==4.47.0
!pip install -q accelerate==0.27.0
!pip install -q peft
!pip install -q bitsandbytes==0.45.0
!pip install -q sacrebleu
!pip install -q unbabel-comet
!pip install -q optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 48.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 279.7/279.7 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.2/96.2 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.4/101.4 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.3/819.3 kB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 529.7/529.7 kB 38.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [4]:
from datasets import load_dataset

data_train = load_dataset("csv", data_files='/content/drive/MyDrive/Colab Notebooks/TA/A3/pt_en_ASR_Whisper_Medium_train.csv')
data_train = data_train.select_columns(["transcription", "translation"])
data_train = data_train["train"].train_test_split(test_size = 0.83, shuffle = False)

Generating train split: 0 examples [00:00, ? examples/s]

In [5]:
data_train["train"][:5]["transcription"]

[' altere o dispositivo de saída de áudio.',
 ' Até a data limite prevista no tratado, vários mices foram destruídos.',
 ' Eu balancei a cabeça.',
 ' Adsoni de Cranberries Zombie Aplicia de clássicas',
 ' Um cinegrafista filmando um jogo de basquete.']

In [6]:
data_train["train"][:5]["translation"]

['Change the audio output device.',
 'By the deadline shown in the treaty, various missiles had been destroyed.',
 'I shook my head.',
 'Add The Cranberries Zombie to the classics playlist.',
 'A cameraman filming a basketball game.']

In [7]:
data_train["test"][:5]["transcription"]

[' O repórter perguntou três vezes seguidas.',
 ' De mais.',
 ' Ela não respondeu nada.',
 ' De qualquer forma, você tem muitas coisas para fazer.',
 ' Aquele que paga por último nunca paga duas vezes.']

In [8]:
data_train["test"][:5]["translation"]

['The reporter asked three times in a row.',
 'Enough!',
 'She didn’t answer anything.',
 'In any case, you have a lot of stuff to do.',
 'The one who pays last never pays twice.']

In [9]:
max_tok_length_train = max(len(sample["transcription"]) for sample in data_train["train"])
max_tok_length_test = max(len(sample["transcription"]) for sample in data_train["test"])

max_tok_length = max(max_tok_length_train, max_tok_length_test)

print(f"Dynamic max_tok_length set to: {max_tok_length}")


from transformers import AutoTokenizer

checkpoint = "facebook/nllb-200-distilled-600M"
src_code = "por_Latn"
tgt_code = "eng_Latn"
tokenizer = AutoTokenizer.from_pretrained(
    checkpoint,
    padding=True,
    pad_to_multiple_of=8,
    src_lang=src_code,
    tgt_lang=tgt_code,
    truncation=False,
    max_length=max_tok_length,
    )

Dynamic max_tok_length set to: 284


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/564 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.3M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/3.55k [00:00<?, ?B/s]

In [10]:
def preprocess_function(sample):
    model_inputs = tokenizer(
        sample["transcription"],
        text_target = sample["translation"],
        )
    return model_inputs


In [11]:
tokenized_data_train = data_train.map(preprocess_function, batched=True)

Map:   0%|          | 0/340 [00:00<?, ? examples/s]

Map:   0%|          | 0/1660 [00:00<?, ? examples/s]

In [12]:
dic = {}
for sample in tokenized_data_train['train']:
    sample_length = len(sample['input_ids'])
    if sample_length not in dic:
        dic[sample_length] = 1
    else:
        dic[sample_length] += 1

for i in range(1, max_tok_length + 1):
    if i in dic:
        print(f"{i:>2} {dic[i]:>3}")

 4   3
 5   4
 6  11
 7  22
 8  29
 9  30
10  38
11  31
12  28
13  25
14  24
15  22
16  19
17   8
18   9
19   3
20  14
21   6
22   7
23   4
24   1
26   1
28   1


In [13]:
import torch
from transformers import BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
)

In [14]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained(
    checkpoint,
    quantization_config=quantization_config
    )


config.json:   0%|          | 0.00/846 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now default to True since model is quantized.


pytorch_model.bin:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

In [15]:
from peft import prepare_model_for_kbit_training

model = prepare_model_for_kbit_training(model, use_gradient_checkpointing=False, gradient_checkpointing_kwargs={'use_reentrant':False})

In [16]:
from evaluate import load

metric = load("sacrebleu")

In [17]:
import numpy as np

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    for i in range(len(labels)):
        labels[i] = [tokenizer.pad_token_id if j<0 else j for j in labels[i]]
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

### **Hyperparameter Search**

In [18]:
import optuna
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq
from sklearn.model_selection import train_test_split
from peft import LoraConfig, get_peft_model

def objective(trial):
    learning_rate = trial.suggest_loguniform("learning_rate", 1e-5, 1e-3)
    batch_size = trial.suggest_categorical("batch_size", [2, 4, 8, 16])
    lora_r = trial.suggest_categorical("lora_r", [8, 16, 32])
    lora_alpha = trial.suggest_int("lora_alpha", 16, 64, step=16)
    lora_dropout = trial.suggest_uniform("lora_dropout", 0.0, 0.3)

    config = LoraConfig(
        task_type="SEQ_2_SEQ_LM",
        r=lora_r,
        lora_alpha=lora_alpha,
        target_modules=["q_proj", "v_proj"],
        lora_dropout=lora_dropout,
        bias="none",
    )

    lora_model = get_peft_model(model, config)

    data_collator = DataCollatorForSeq2Seq(
                      tokenizer=tokenizer,
                      model=lora_model,
                      pad_to_multiple_of=8
                      )

    args = Seq2SeqTrainingArguments(
        output_dir="./optuna_results",
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        learning_rate=learning_rate,
        num_train_epochs=3,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        save_total_limit=2,
        predict_with_generate=True,
        logging_dir="./logs",
        logging_strategy="epoch",
        load_best_model_at_end=True,
        metric_for_best_model="eval_loss",
    )

    trainer = Seq2SeqTrainer(
        lora_model,
        args,
        train_dataset=tokenized_data_train["train"],
        eval_dataset=tokenized_data_train["test"],
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
    )

    trainer.train()
    eval_results = trainer.evaluate()
    return eval_results["eval_loss"]


In [19]:
study = optuna.create_study(direction = "minimize")
study.optimize(objective, n_trials = 5)

Output hidden; open in https://colab.research.google.com to view.

In [42]:
print("Best hyperparameters: ", study.best_params)

Best hyperparameters:  {'learning_rate': 0.00031379463096667376, 'batch_size': 2, 'lora_r': 32, 'lora_alpha': 16, 'lora_dropout': 0.09819107823091107}


In [20]:
best_params = study.best_params

final_lora_config = LoraConfig(
    task_type="SEQ_2_SEQ_LM",
    r=best_params["lora_r"],
    lora_alpha=best_params["lora_alpha"],
    target_modules=["q_proj", "v_proj"],
    lora_dropout=best_params["lora_dropout"],
    bias="none",
)

In [21]:
final_model = get_peft_model(model, final_lora_config)
final_model.print_trainable_parameters()

trainable params: 4,718,592 || all params: 619,792,384 || trainable%: 0.7613


In [22]:
data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=final_model,
    pad_to_multiple_of=8
    )

### **Training**

In [23]:
final_args = Seq2SeqTrainingArguments(
    output_dir="./final_model",
    run_name = 'best_model_run',
    per_device_train_batch_size=best_params["batch_size"],
    per_device_eval_batch_size=best_params["batch_size"],
    learning_rate=best_params["learning_rate"],
    num_train_epochs=5,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    predict_with_generate=True,
    logging_dir="./final_logs",
    load_best_model_at_end=True,
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [24]:
final_trainer = Seq2SeqTrainer(
    final_model,
    final_args,
    train_dataset=tokenized_data_train["train"],
    eval_dataset=tokenized_data_train["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)


<ipython-input-24-3bce10d72d4c>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  final_trainer = Seq2SeqTrainer(


In [25]:
final_trainer.train()

Epoch,Training Loss,Validation Loss,Bleu,Gen Len
1,No log,0.970397,49.342300,12.400600
2,No log,0.830963,51.127600,12.431900
3,0.875100,0.831496,51.066400,12.484300
4,0.875100,0.840302,51.187200,12.558400
5,0.875100,0.840772,51.110600,12.544600


Se han truncado las últimas 5000 líneas del flujo de salida.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class in

TrainOutput(global_step=850, training_loss=0.7721002017750459, metrics={'train_runtime': 3904.3636, 'train_samples_per_second': 0.435, 'train_steps_per_second': 0.218, 'total_flos': 66054438912000.0, 'train_loss': 0.7721002017750459, 'epoch': 5.0})

In [26]:
from transformers import GenerationConfig

generation_config = GenerationConfig.from_pretrained(
    checkpoint,
)

print(generation_config)

GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "eos_token_id": 2,
  "max_length": 200,
  "pad_token_id": 1
}



### **Load Test Set**

In [27]:
from datasets import load_dataset

data_test = load_dataset("csv", data_files='/content/drive/MyDrive/Colab Notebooks/TA/A3/pt_en_ASR_Whisper_Medium_test.csv')

data_test

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'transcription', 'sentence', 'translation', 'transcription_clean', 'sentence_clean', 'translation_clean'],
        num_rows: 1000
    })
})

In [28]:
def preprocess_function(sample):
    model_inputs = tokenizer(
        sample["transcription"],
        text_target = sample["translation"],
        )
    return model_inputs

In [29]:
tokenized_data_test = data_test.map(preprocess_function, batched=True)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [30]:
test_batch_size = 32
batch_tokenized_test = tokenized_data_test['train'].batch(test_batch_size)

Batching examples:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [31]:
number_of_batches = len(batch_tokenized_test["transcription"])
output_sequences = []
for i in range(number_of_batches):
    inputs = tokenizer(batch_tokenized_test["transcription"][i],
                       max_length=max_tok_length, truncation=False,
                       return_tensors="pt", padding=True)
    output_batch = final_model.generate(generation_config=generation_config,
                                        input_ids=inputs["input_ids"].cuda(),
                                        attention_mask=inputs["attention_mask"].cuda(),
                                        forced_bos_token_id=tokenizer.convert_tokens_to_ids(tgt_code),
                                        max_length = max_tok_length, num_beams=1, do_sample=False,)
    output_sequences.extend(output_batch.cpu())

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:2691: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


In [32]:
decoded_preds = tokenizer.batch_decode(output_sequences, skip_special_tokens=True)

In [33]:
references = tokenizer.batch_decode(tokenized_data_test["train"]["labels"], skip_special_tokens=True)

In [34]:
references[:5]

['Borrow money from people in the village',
 'Lock them up',
 'Youtube is still the best video platform.',
 'A girl and a boy kissing in the shadows',
 'I actually prefer to eat my sliced couscous as it is, instead of grilled.']

In [35]:
decoded_preds[:5]

['And money borrowed from the people of the village.',
 'We locked up.',
 'YouTube is still the best video platform.',
 'Girl and boy kissing in the shadows',
 'Particularly my cut cushion I like to eat well instead of cattle.']

In [36]:
from whisper.normalizers.basic import BasicTextNormalizer

normalizer = BasicTextNormalizer()

decoded_preds_clean = [normalizer(text) for text in decoded_preds]
references_clean = [normalizer(text) for text in references]

In [37]:
from evaluate import load

metric = load("sacrebleu")

In [38]:
result = metric.compute(predictions=decoded_preds_clean, references=references_clean)
print(f'BLEU score: {result["score"]:.1f}')

BLEU score: 53.3


In [39]:
from evaluate import load
comet_metric = load('comet')

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

LICENSE:   0%|          | 0.00/9.69k [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.38k [00:00<?, ?B/s]

hparams.yaml:   0%|          | 0.00/567 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/pytorch_lightning/core/saving.py:195: Found keys that are not in the model state dict but in the checkpoint: ['encoder.model.embeddings.position_ids']


In [40]:
comet_score = comet_metric.compute(predictions=decoded_preds_clean, references=references_clean, sources=data_test["train"]["transcription"])

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


In [41]:
print(f"COMET: {comet_score['mean_score'] * 100:.2f} %")

COMET: 86.97 %
